# pyarrow 방식 성능 테스트  
- cache 누적 처리 시 성능 저하 문제 회피  
- 메모리 적재 형태 데이터를 spark로 전달하고 처리 후 다시 pandas 형태로 처리하는 테스트   
- hdfs data는 미리 접근 가능한 위치로 옮기고 처리한다 가정...  


In [1]:
# 필요 라이브러리 임포트  
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from os.path import abspath
# import findspark
import time 
import numpy as np 
import pandas as pd
import pyarrow 

In [50]:
# !pip install findspark

In [2]:
import findspark
findspark.init()

In [62]:
# sys.version
# pd.__version__
import pyspark
pyspark.__version__
# !pip install pandas==1.3.0
pyarrow.__version__

'6.0.1'

In [3]:
# 환경변수 정의  
scale = 1000 # 1000 만 건 수준
PRJ_ROOT = '/user/root'
APP_NAME = 'RDD-Pandas'
DB_NAME = 'inven'

# 데이터의 파일 포맷 및 파일명  
tbl_setop_name = 'inven/table-set-6m-20-1000'
file_format = 'parquet' 


In [4]:
# 스파크 생성 
def spark_creation():
    spark = SparkSession.builder.master('yarn').appName(APP_NAME)\
    .config('spark.rpc.message.maxSize', '1024')\
    .config('spark.sql.execution.arrow.enabled', 'true')\
    .config('spark.sql.execution.arrow.pyspark.enabled', 'true')\
    .config('spark.sql.execution.arrow.pyspark.fallback.enabled', 'true')\
    .config('spark.driver.cores', '1').config('spark.driver.memory', '7g')\
    .config('spark.num.executors', '3')\
    .config('spark.executor.cores', '1').config('spark.executor.memory', '7g')\
    .config('spark.jars', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar')\
    .config('spark.driver.extraClassPath', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar').getOrCreate()
    #     .config('spark.sql.execution.arrow.enabled', 'true')\
    # spark.rpc.message.maxSize  240007497 
    sc = spark.sparkContext
    sc
    return spark

In [5]:
%%time
spark = spark_creation()
spark

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


CPU times: user 32.1 ms, sys: 16 ms, total: 48.1 ms
Wall time: 18.6 s


In [7]:
# !pip install pyarrow==7.0.0

In [6]:
%%time
# df = pyarrow.parquet.read_table(source="out_parq").to_pandas()
# df
import pyarrow.parquet as pq
adf = pq.read_table("parq")
print(adf.shape)

(10000000, 20)
CPU times: user 792 ms, sys: 587 ms, total: 1.38 s
Wall time: 1.16 s


In [7]:
%%time
# 400 만 : 12 초  
pdf = adf.to_pandas()
sdf = spark.createDataFrame(pdf)
sdf.show(1)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/py4j/clientserver.py", line 480, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.6/dist-packages/py4j/clientserver.py", line 504, in send_command
    "Error while sending or receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o90.showString

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/py4j/clientserver.py", line 480, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.6/dist-packages/py4j/clientserver.py", line 504, in send_command
    "Error while sending or receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [43]:
# %%time
# # sdf 를 pandas로   
# sdf.select("setop").show()


In [17]:
%%time
from pyspark.sql.functions import col
# sdf.select("setop").toPandas()
# sdf.withColumn("setop_new", col("setop")).select("setop").toPandas()
sdf.printSchema()
sdf.show(truncate=False)
sdf.toPandas()

root
 |-- setop: string (nullable = true)
 |-- stype: string (nullable = true)
 |-- inv_rate_01: float (nullable = true)
 |-- inv_val_01: long (nullable = true)
 |-- inv_req_01: long (nullable = true)
 |-- inv_rate_02: float (nullable = true)
 |-- inv_val_02: long (nullable = true)
 |-- inv_req_02: long (nullable = true)
 |-- inv_rate_03: float (nullable = true)
 |-- inv_val_03: long (nullable = true)
 |-- inv_req_03: long (nullable = true)
 |-- inv_rate_04: float (nullable = true)
 |-- inv_val_04: long (nullable = true)
 |-- inv_req_04: long (nullable = true)
 |-- inv_rate_05: float (nullable = true)
 |-- inv_val_05: long (nullable = true)
 |-- inv_req_05: long (nullable = true)
 |-- inv_rate_06: float (nullable = true)
 |-- inv_val_06: long (nullable = true)
 |-- inv_req_06: long (nullable = true)

+------------+-----+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+---------

/usr/local/lib/python3.6/dist-packages/pyspark/sql/pandas/conversion.py:153: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true, but has reached the error below and can not continue. Note that 'spark.sql.execution.arrow.pyspark.fallback.enabled' does not have an effect on failures in the middle of computation.
  An error occurred while calling o155.arrowPySparkSelfDestructEnabled. Trace:
py4j.Py4JException: Method arrowPySparkSelfDestructEnabled([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)


  warnings.warn(msg)


Py4JError: An error occurred while calling o155.arrowPySparkSelfDestructEnabled. Trace:
py4j.Py4JException: Method arrowPySparkSelfDestructEnabled([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)



In [63]:
# long 컬럼 제외하고 pd로 만들어서 다시 테스트  
pdf = pd.DataFrame()
pdf["setop"] = ["A", "B", "C"]
pdf["val"] = [1, 2, 3]
pdf

,setop,val
0,A,1
1,B,2
2,C,3


In [64]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, FloatType, IntegerType
columns = [
    StructField("setop", StringType())
    , StructField("val", IntegerType())]
sample_schema = StructType(columns)

sdf = spark.createDataFrame(pdf, sample_schema)
sdf.show()

+-----+---+
|setop|val|
+-----+---+
|    A|  1|
|    B|  2|
|    C|  3|
+-----+---+



In [65]:
sdf.printSchema()
sdf.show(truncate=False)
sdf.toPandas()

root
 |-- setop: string (nullable = true)
 |-- val: integer (nullable = true)

+-----+---+
|setop|val|
+-----+---+
|A    |1  |
|B    |2  |
|C    |3  |
+-----+---+



,setop,val
0,A,1
1,B,2
2,C,3


In [1]:
spark.stop()

NameError: name 'spark' is not defined